In [1]:
# -------- Make sure you have opencv-python installed in you environment -------- #
import cv2
import fnmatch, os
import numpy as np
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [24]:
# -------- Not critical if file dividing is not your major concern -------- #
def return_portion(arr, k):
    arr_flat = arr.reshape((-1,1))
    if k <= arr_flat[-1] and k >= arr_flat[0]:
        for i in range(len(arr_flat)):
            if arr_flat[i]>k:
                break
        p = np.where(arr == arr_flat[i-1])[0]
    else:
        p = -1
    return int(p)

def return_timestamps(tree):
    ts = []
    num_frames = 0
    for elem in tree.iter():
        if elem.tag =='frame' and num_frames > 0:
            ts.append(int(elem.attrib['timestamp']))
        elif elem.tag == 'frame' and num_frames == 0:
            num_frames = 1
    return np.asarray(ts)

def file_divide(ts):
    threshold = 33333*8
    diff_ts = np.diff(ts)
    frame_numbers = np.where(diff_ts > threshold)[0]
    frame_start, frame_end = [], []
    for i in range(len(frame_numbers)):
        frame_start.append(frame_numbers[i]+1)
        if i < len(frame_numbers)-1:
            frame_end.append(frame_numbers[i+1])
        else:
            frame_end.append(len(ts))
    temp = [frame_start,frame_end]
    return np.asarray(temp).T

base_path = r'D:\Lab\Data\DLC_videos\Han_20201204_rwFreeReach\Han_reaching-Joe-2020-12-10\videos\calib\\'
save_path = base_path
info_type = '*.xiinfo'
info_file_list = fnmatch.filter(os.listdir( base_path ), info_type)
info_file_list = np.sort(info_file_list)

"""
The next 3 lines will find out all .avi files under a specific path.
Typically they are the files from different cameras
"""
video_type = '*trimmed.avi'
video_file_list = fnmatch.filter(os.listdir( base_path ), video_type)
video_file_list = np.sort(video_file_list)

# get timestamps from info files for annotating videos
ts_list = []
for each in info_file_list:
    tree = ET.ElementTree(file = ''.join( (base_path, each) ))
    ts_list.append(return_timestamps(tree))

"""The number of frames for file dividing with each of the 3 cameras are stored in this list """
seperate_frames = []
for each in ts_list:
    frames = file_divide(each)
    seperate_frames.append(frames)
#%% -----------------------------------------------------------------------------
#--------------------------------------------------------------------------------
"""
`s` defines whether you want to use a smaller size
When `s = 1`, you have the same frame size as you original videos
When `s = 2`, you have only the half size, e.g., your orignial video is 1280*1024, the generated video will only have 640*512
"""
s = 1

"""
Put the video you want to process in this list
"""
print(video_file_list)

['Han_20201204_00001.avi_trimmed.avi']


In [18]:
"""
Set what time bounds to keep frames. Convert time bounds to frames and store frame numbers
"""
#time_bounds = np.array([[0,449],[465,859],[890,986]]) # in seconds
time_bounds = np.array([[0,1]]) # in seconds


In [26]:
video_file_list_idx = [0]
running_list = video_file_list[video_file_list_idx]

add_frame_nums = 1
appended_text = '_trimmed.avi'
"""
Specify the frame rate here. Typical value is 30 frames per second
"""

"""
The color of the annotation you want to add in each frame. Here is red
"""
color=(0,0,255)

for (i, each) in enumerate(running_list):
    # -- Basic setup of opencv -- #
    cap = cv2.VideoCapture(base_path + each)
    fps = int(round(cap.get(cv2.CAP_PROP_FPS)))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    save_fps = fps
    frame_bounds = time_bounds*fps
    frame_bounds = np.array([[-1,150]])
    """
    The main idear here is to let opencv read in individual frames,
    process them and then write to a new video file
    """

    # -- If file dividing is not your major concern, you may only need one video_writer --#
    video_writers = []
    new_video = each[:-4] + appended_text
    video_writers.append(cv2.VideoWriter(save_path + new_video,fourcc, save_fps, (int(width/s), int(height/s))) )
    
    frame_list = []
    timestamp_list = []
    # -- Loop until all frames are done -- #
    for k in range(0, total_frames):
        if k % 400 == 0:
            print(k)
        # -- Read one video frame -- #
        cap.set(cv2.CAP_PROP_POS_FRAMES,k-1)
        success, frame = cap.read()
        writer_no = 0
        
        # -- Write one video frame if we are supposed to keep it -- #
        if time_bounds.size == 0:
            keep_frame = 1
        else:
            keep_frame = 0
            for i_bound in range(0,frame_bounds.shape[0]):
                if(k >= frame_bounds[i_bound,0] and k <= frame_bounds[i_bound,1]):
                    keep_frame = 1
        
        if success and keep_frame:
            # -- Resize the frame based on the variable `s` defined before -- #
            #frame = cv2.resize(frame, (int(width/s), int(height/s)))
            start_frame = 0
            ####################################################################
            # Add the annotations of frame number and timestamp in seconds using the putText function            
            ####################################################################
            if(add_frame_nums):
                cv2.putText(frame, str(k-start_frame+1), (20, 20) ,cv2.FONT_HERSHEY_SIMPLEX ,0.8, color,2)
                #t = (ts_list[video_file_list_idx[i]][k] - ts_list[video_file_list_idx[i]][start_frame])/1000000
                #cv2.putText(frame, str(round(t, 3)), (120, 20) ,cv2.FONT_HERSHEY_SIMPLEX ,0.8, color,2)
                
            # -- Write to the video file -- #
            frame_list.append(k)
            timestamp_list.append(ts_list[video_file_list_idx[i]][k])
            video_writers[writer_no].write(frame)
        elif not success:
            print("failed to read")
    cap.release()
    video_writers[writer_no].release()
    
    # write frame_list and timestamp_list to a txt file
    np.savetxt(save_path+each[:-4]+'_trimmed.txt',np.transpose(np.array([frame_list,timestamp_list])),
          fmt=['%.0f','%.0f'])
    
    
    print('end for the No. %d big file'%(i))
                
    

0
end for the No. 0 big file


In [20]:
frame_bounds

array([[ 0, 24]])